In [2]:
"""
Created on Mon Aug 27 16:04:09 2018

@author: Ravi Kamble
"""

'\nCreated on Mon Aug 27 16:04:09 2018\n\n@author: Ravi Kamble\n'

In [6]:
import sys 
import os
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
import keras
from keras.callbacks import TensorBoard
from keras.applications.mobilenet import MobileNet

# fix seed for reproducible results (only works on CPU, not GPU)
seed = 9
np.random.seed(seed=seed)
tf.set_random_seed(seed=seed)

In [25]:
# All defined hyper parameters for model
nb_classes = 2                             # number of classes
based_model_last_block_layer_number = 126  # value is based on based model selected.
img_width, img_height = 224, 224           # change based on the shape/structure of your images for squeezenet
batch_size = 32                            # dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 20                              # number of iteration the algorithm gets trained.
learn_rate = 1e-4                          # sgd learning rate
momentum = .9                              # sgd momentum to avoid local minimum
transformation_ratio = .05                 # how aggressive will be the data augmentation/transformation
nb_train_samples = 240                     # Total number of train samples. NOT including augmented images
nb_validation_samples = 96                 # Total number of train samples. NOT including augmented images.

In [26]:
def train(train_data_dir, validation_data_dir, model_path):
    # Pre-Trained CNN Model using imagenet dataset for pre-trained weights
       
    if K.image_data_format() == 'channels_first': 
        input_shape = (3, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 3)
    
    # We use Xception as it gives more stable performance
    base_model = Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)  
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)

    predictions = Dense(nb_classes, activation='softmax', name='predictions')(x)

    # add the top layer block to your base model
    model = Model(base_model.input, predictions)
    print(model.summary())

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all layers of the based model that is already pre-trained.
    for layer in base_model.layers:
        layer.trainable = False

    # Read Data and Augment it: Make sure to select augmentations that are appropriate to your images.
    # To save augmentations un-comment save lines and add to your flow parameters.
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=transformation_ratio,
                                       shear_range=transformation_ratio,
                                       zoom_range=transformation_ratio,
                                       cval=transformation_ratio,
                                       horizontal_flip=True,
                                       vertical_flip=True)

    validation_datagen = ImageDataGenerator(rescale=1. / 255)

    # os.makedirs(os.path.join(os.path.abspath(train_data_dir), 'preview'), exist_ok=True)
    train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                        target_size=[img_width, img_height],
                                                        batch_size=batch_size,
                                                        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                                  target_size=[img_width, img_height],
                                                                  batch_size=batch_size,
                                                                  class_mode='categorical')

    model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
                  metrics=['accuracy'])

    # save weights of best training epoch: monitor either val_loss or val_acc

    top_weights_path = os.path.join(os.path.abspath(model_path), 'top_model_weights.h5')
    callbacks_list = [
        ModelCheckpoint(top_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        #EarlyStopping(monitor='val_acc', patience=5, verbose=0),
        keras.callbacks.TensorBoard(log_dir='tensorboard/inception-v3-train-top-layer', histogram_freq=0, write_graph=False, write_images=False)
    ]

    # Train Simple CNN
    model.fit_generator(train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=nb_epoch / 5,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size,
                        callbacks=callbacks_list)

    # verbose
    print("\nStarting to Fine Tune Model\n")

    # add the best weights from the train top model
    # at this point we have the pre-train weights of the base model and the trained weight of the new/added top model
    # we re-load model weights to ensure the best epoch is selected and not the last one.
    model.load_weights(top_weights_path)

    for layer in model.layers[:based_model_last_block_layer_number]:
        layer.trainable = False
    for layer in model.layers[based_model_last_block_layer_number:]:
        layer.trainable = True

    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
                  
    
    # save weights of best training epoch: monitor either val_loss or val_acc
    final_weights_path = os.path.join(os.path.abspath(model_path), 'model_weights.h5')
    callbacks_list = [
        ModelCheckpoint(final_weights_path, monitor='val_acc', verbose=1, save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=5, verbose=0),
        keras.callbacks.TensorBoard(log_dir='tensorboard/inception-v3-fine-tune', histogram_freq=0, write_graph=False, write_images=False)
    ]
    
 
    newpath = 'G:/RAVISIR/Hello'    # Path for storing tensorboard file 
    tensorboard = TensorBoard(log_dir = newpath)


    # fine-tune the model
    model.fit_generator(train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=nb_epoch,
                        validation_data=validation_generator,
                        validation_steps=nb_validation_samples // batch_size,
                        callbacks=[tensorboard])

    # save model
    model_json = model.to_json()
    with open(os.path.join(os.path.abspath(model_path), 'model.json'), 'w') as json_file:
        json_file.write(model_json)

# Path for Train & validation dataset. Give this folder path to run again 

train(r'G:\RAVISIR\dataset1\train', r'G:\RAVISIR\dataset1\validation', '.')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_12[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

Epoch 1/4
7/7 [==============================] - 5s 701ms/step - loss: 0.7765 - acc: 0.6920 - val_loss: 0.0470 - val_acc: 0.9896

Epoch 00001: val_acc improved from -inf to 0.98958, saving model to C:\Users\COE1\top_model_weights.h5
Epoch 2/4
7/7 [==============================] - 4s 531ms/step - loss: 0.4096 - acc: 0.8026 - val_loss: 0.1531 - val_acc: 0.9271

Epoch 00002: val_acc did not improve
Epoch 3/4
7/7 [==============================] - 3s 494ms/step - loss: 0.0968 - acc: 0.9730 - val_loss: 0.0360 - val_acc: 0.9896

Epoch 00003: val_acc did not improve
Epoch 4/4
7/7 [==============================] - 3s 486ms/step - loss: 0.0924 - acc: 0.9742 - val_loss: 0.4723 - val_acc: 0.8333

Epoch 00004: val_acc did not improve

Starting to Fine Tune Model

Epoch 1/20
7/7 [==============================] - 5s 768ms/step - loss: 0.8631 - acc: 0.8648 - val_loss: 0.0850 - val_acc: 0.9688
Epoch 2/20
7/7 [==============================] - 4s 593ms/step - loss: 0.0181 - acc: 0.9910 - val_loss: 0